In [ ]:
function free_final_time_discrete_dynamics!(x_dot, x,
                p::Tuple{Any, Any, Float64, Int64, AbstractModel, NTuple{15, Any}}, tau)
                control, tau_ref, tf, k, model, (A, B, STM, STM_inv, ẋ, Ȧ, Bṁ, Bṗ, Ṡ, ż, aux1, aux2, aux3, aux4, aux5) = p

    n = length(model.x0)
    n1 = n + n^2

    x_ref = @view x[1:n]
    u_ref = get_control(control, tau*tf)

    getAB!(A, B, x_ref, model)
    A .*= tf
    B .*= tf

    # state dynamics
    dynamics!(aux1, x_ref, (control, model), tau*tf)
    ẋ .= aux1 .* tf

    # STM
    STM .= reshape((@view x[n+1:n1]), (n,n))
    STM_inv .= inv(STM)
    mul!(Ȧ, A, STM)

    # B minus matrix
    lambda_minus = (tau_ref[k+1]-tau)/(tau_ref[k+1]-tau_ref[k])
    aux2 .= STM_inv .* lambda_minus
    mul!(Bṁ, aux2, B)

    # B plus matrix
    lambda_plus = (tau - tau_ref[k])/(tau_ref[k+1]-tau_ref[k])
    aux3 .= STM_inv .* lambda_plus
    mul!(Bṗ, aux3, B)

    # S matrix
    mul!(Ṡ, STM_inv, aux1)
    # dynamics!(Ṡ, x_ref, (control, model), tau*tf)

    # z vector 
    B .*= u_ref
    mul!(aux4, A, x_ref)
    aux5 .= aux4 .+ B
    mul!(ż, STM_inv, aux5)
    ż .*= -1.0

    x_dot .= [ẋ; reduce(vcat, Ȧ); Bṁ; Bṗ; Ṡ; ż]

    nothing
end